In [73]:
import json
import pandas as pd

In [74]:
# Create File Path
filepath = "C:/Users/piercerachel/Desktop/git/Project2_GroupC/"
filename = "monster_india_latest_jobs_free_dataset.json"

# Opening JSON file
f = open(filepath + filename)

# returns JSON object as a dictionary
data = json.load(f)

In [75]:
# Create data list
address_country = []
address_locality = []
address_region = []
company = []
description = []
industry = []
postal_code = []
posted_at = []
salary = []  # the target
salary_type = []
skills = []
title = []
url = []

# iterate through each element in the data list
for each in data:

    # append values to list, given each key
    address_country.append(each["address_country"])
    address_locality.append(each["address_locality"])
    address_region.append(each["address_region"])
    company.append(each["company"])
    description.append(each["description"])
    industry.append(each["industry"])
    postal_code.append(each["postal_code"])
    posted_at.append(each["posted_at"])
    salary.append(each["salary"])
    salary_type.append(each["salary_type"])
    skills.append(each["skills"]) 
    title.append(each["title"])
    url.append(each["url"])

In [76]:
# create df, using string header and list as rows
df = pd.DataFrame({"address_country": address_country,
                   "address_locality": address_locality,
                   "address_region": address_region,
                   "company": company,
                   "description": description,
                   "industry": industry,
                   "postal_code": postal_code,
                   "posted_at": posted_at,
                   "salary": salary,
                   "salary_type": salary_type,
                   "skills": skills,
                   "title": title,
                   "url": url,
                  })

In [77]:
# Preview dataframe
df.head()

,address_country,address_locality,address_region,company,description,industry,postal_code,posted_at,salary,salary_type,skills,title,url
0,IN,India,India,Kotak Mahindra Bank Limited,Job Description : - . Client Engagement o...,Banking/Accounting/Financial Services,NA,2021-09-16,Not disclosed,P.A.,Service Relationship Manager-POS-BRANCH BANKIN...,Service Relationship Manager-POS-BRANCH BANKIN...,https://www.monsterindia.com/job/service-relat...
1,IN,Noida,Uttar Pradesh,ANI Calls India Private Limited,Job Description : . Axway B2Bi or Axway Gatew...,IT/Computers - Software,NA,2021-09-13,1600000,P.A.,Axway B2B EDI Integration consultant,Axway B2B EDI Integration consultant,https://www.monsterindia.com/job/axway-b2b-edi...
2,IN,Chennai,Tamil Nadu,PayPal,Job Description : Job Description Fueled by a...,Banking/Accounting/Financial Services,NA,2021-08-18,Not disclosed,P.A.,Data Engineer/MTS 2,"MTS 2, Data Engineer",https://www.monsterindia.com/job/mts-2-data-en...
3,IN,Bengaluru / Bangalore,Karnataka,ANI Calls India Private Limited,Job Description : At least three years WebLog...,IT/Computers - Software,NA,2021-08-26,1800000,P.A.,Senior Oracle Fusion Middleware developer,Senior Oracle Fusion Middleware developer,https://www.monsterindia.com/job/senior-oracle...
4,IN,India,India,ANI Calls India Private Limited,Job Description : • Linux engineer with ksh s...,IT/Computers - Software,NA,2021-09-08,1400000,P.A.,Korn Shell (KSH) - Linux System Engineer,Korn Shell (KSH) - Linux System Engineer,https://www.monsterindia.com/job/korn-shell-ks...


In [78]:
# Clean data, drop unnecessary columns
df.drop(['address_country', 'address_locality', 'address_region', 'postal_code', 'salary_type', 'skills', 'url'], axis=1, inplace=True)

In [80]:
df.head()

,company,description,industry,posted_at,salary,title
0,Kotak Mahindra Bank Limited,Job Description : - . Client Engagement o...,Banking/Accounting/Financial Services,2021-09-16,Not disclosed,Service Relationship Manager-POS-BRANCH BANKIN...
1,ANI Calls India Private Limited,Job Description : . Axway B2Bi or Axway Gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
2,PayPal,Job Description : Job Description Fueled by a...,Banking/Accounting/Financial Services,2021-08-18,Not disclosed,"MTS 2, Data Engineer"
3,ANI Calls India Private Limited,Job Description : At least three years WebLog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,Job Description : • Linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer


In [81]:
# Drop any salaries that are Not disclosed
df = df[df["salary"] != "Not disclosed"]
df.head()

,company,description,industry,posted_at,salary,title
1,ANI Calls India Private Limited,Job Description : . Axway B2Bi or Axway Gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant
3,ANI Calls India Private Limited,Job Description : At least three years WebLog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer
4,ANI Calls India Private Limited,Job Description : • Linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer
5,ANI Calls India Private Limited,Job Description : . Oracle EBS Apps DBA Imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure
7,Live Connections,Requirement Mulesoft Developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer


In [82]:
# Check if there are any NAs/nulls
df.isnull().sum()

company        0
description    0
industry       0
posted_at      0
salary         0
title          0
dtype: int64

In [83]:
# Add columns for skills learned in Bootcamp
df["SQL"] = 0
df["Python"] = 0
df["Pandas"] = 0
df["API"] = 0
df["Machine Learning"] = 0
df["Artificial Intelligence"] = 0
df["Blockchain"] = 0
df["Solidity"] = 0
df["Financial Modeling"] = 0
df["Algorithmic Trading"] = 0

In [84]:
df.head()

,company,description,industry,posted_at,salary,title,SQL,Python,Pandas,API,Machine Learning,Artificial Intelligence,Blockchain,Solidity,Financial Modeling,Algorithmic Trading
1,ANI Calls India Private Limited,Job Description : . Axway B2Bi or Axway Gatew...,IT/Computers - Software,2021-09-13,1600000,Axway B2B EDI Integration consultant,0,0,0,0,0,0,0,0,0,0
3,ANI Calls India Private Limited,Job Description : At least three years WebLog...,IT/Computers - Software,2021-08-26,1800000,Senior Oracle Fusion Middleware developer,0,0,0,0,0,0,0,0,0,0
4,ANI Calls India Private Limited,Job Description : • Linux engineer with ksh s...,IT/Computers - Software,2021-09-08,1400000,Korn Shell (KSH) - Linux System Engineer,0,0,0,0,0,0,0,0,0,0
5,ANI Calls India Private Limited,Job Description : . Oracle EBS Apps DBA Imple...,IT/Computers - Software,2021-09-16,1400000,Oracle Cloud Infrastructure,0,0,0,0,0,0,0,0,0,0
7,Live Connections,Requirement Mulesoft Developer,Recruitment/Staffing/RPO,2021-08-30,2500000,Mulesoft Developer,0,0,0,0,0,0,0,0,0,0


In [ ]:
# If description does not include keywords (skills) then drop row.  
# Perhaps if description contains SQL, db, database, etc (to be less specific)

In [ ]:
# Objective - if description column contains keyword then 1, if not 0.  If SQL is in description then 1, else 0.
# Next step - salary = y, dummy columns (SQL, etc) are x, split data, train on first 70% of observations with number regression
# Run model on test, see if it performs
# Regression - Predict salary based on skills learned in class OR Classification - cutoff point for salary
# Optional Chatbot of what skills do you possess = salary range in rupees